# HW3.2: Neural Transition-Based Dependency Parsing


In this exercise, you are going to build a deep learning model for Neural Networks Transition-Based Dependency Parsing. A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between “head” words and words which modify those heads. Your implementation will be a transition-based parser, which incrementally builds up a parse one step at a time.

To complete this exercise, you will need to complete the code and build a deep learning model for dependency parsing. 

We provide the code for data preparation and the skeleton for PartialParse class. You do not need to understand the code outside of this notebook. 


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import shutil
shutil.copy("/content/drive/MyDrive/FRA 501 IntroNLP&DL/Dataset/HW3-2.zip", "/content/HW3-2.zip")
!unzip -q HW3-2.zip

replace data/dev.conll? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace data/dev.gold.conll? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace data/dev.gold.conll? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## 1. Transition-Based Dependency Parsing

Your implementation will be a transition-based parser, which incrementally builds
up a parse one step at a time. At every step it maintains a partial parse, which is represented as follows:
- A stack of words that are currently being processed.
- A buffer of words yet to be processed.
- A list of dependencies predicted by the parser.

Initially, the stack only contains ROOT, the dependencies lists is empty, and the buffer contains all words
of the sentence in order. At each step, the parse applies a transition to the partial parse until its buffer is
empty and the stack is size 1. The following transitions can be applied:
- SHIFT: removes the first word from the buffer and pushes it onto the stack.
- LEFT-ARC: marks the second (second most recently added) item on the stack as a dependent of the
first item and removes the second item from the stack.
- RIGHT-ARC: marks the first (most recently added) item on the stack as a dependent of the second
item and removes the first item from the stack.

Your parser will decide among transitions at each state using a neural network classifier.

### TODO 1 (Written):
Go through the sequence of transitions needed for parsing the sentence “I parsed
this sentence correctly”. The dependency tree for the sentence is shown below. At each step, give the
configuration of the stack and buffer, as well as what transition was applied this step and what new
dependency was added (if any). The first three steps are provided below as an example.

Image --> https://drive.google.com/file/d/10jYgxDhsyolZGarcNTEdt6G2xB0l9iZU/view?usp=share_link 

Complete the following table (double click the table and fill in the rest):

| stack    |  buffer |  new dependency | transition |
| :------: |:------: | :-------------: | :--------: |
| \[ROOT\]            | \[I, parsed, this, sentence, correctly\] |          | Initial Configuration |
| \[ROOT, I\]         | \[parsed, this, sentence, correctly\]    |          | SHIFT |
| \[ROOT, I, parsed\] | \[this, sentence, correctly\]            |          | SHIFT |
| \[ROOT, parsed\]    | \[this, sentence, correctly\]            | parsed→I | LEFT-ARC |
| | | | |
| | | | |
| | | | |
| | | | |

### TODO 2 (Coding):
Implement the __\_\_init\_\___ and __parse_step__ functions in the PartialParse class. Your code must past both of the following tests.

In [ ]:
class PartialParse(object):
    def __init__(self, sentence):
        """Initializes this partial parse.

        Your code should initialize the following fields:
            self.stack: The current stack represented as a list with the top of the stack as the
                        last element of the list.
            self.buffer: The current buffer represented as a list with the first item on the
                         buffer as the first item of the list
            self.dependencies: The list of dependencies produced so far. Represented as a list of
                    tuples where each tuple is of the form (head, dependent).
                    Order for this list doesn't matter.

        The root token should be represented with the string "ROOT"

        Args:
            sentence: The sentence to be parsed as a list of words.
                      Your code should not modify the sentence.
        """
        # The sentence being parsed is kept for bookkeeping purposes. Do not use it in your code.
        self.sentence = sentence #--list

        ### YOUR CODE HERE
        #self.stack = ?  --> list
        #self.buffer = ? --> list
        #self.dependencies = ?  --> list
        ### END YOUR CODE

    def parse_step(self, transition):
        """Performs a single parse step by applying the given transition to this partial parse

        Args:
            transition: A string that equals "S", "LA", or "RA" representing the shift, left-arc,
                        and right-arc transitions. You can assume the provided transition is a legal
                        transition.
        """
        ### YOUR CODE HERE
        ### END YOUR CODE

    def parse(self, transitions):
        """Applies the provided transitions to this PartialParse

        Args:
            transitions: The list of transitions in the order they should be applied
        Returns:
            dependencies: The list of dependencies produced when parsing the sentence. Represented
                          as a list of tuples where each tuple is of the form (head, dependent)
        """
        for transition in transitions:
            self.parse_step(transition)
        return self.dependencies

In [ ]:
# Do not modify this code
def test_step(name, transition, stack, buf, deps,
              ex_stack, ex_buf, ex_deps):
    """Tests that a single parse step returns the expected output"""
    pp = PartialParse([])
    pp.stack, pp.buffer, pp.dependencies = stack, buf, deps

    pp.parse_step(transition)
    stack, buf, deps = (tuple(pp.stack), tuple(pp.buffer), tuple(sorted(pp.dependencies)))
    assert stack == ex_stack, \
        "{:} test resulted in stack {:}, expected {:}".format(name, stack, ex_stack)
    assert buf == ex_buf, \
        "{:} test resulted in buffer {:}, expected {:}".format(name, buf, ex_buf)
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)
    print("{:} test passed!".format(name))


def test_parse_step():
    """Simple tests for the PartialParse.parse_step function
    Warning: these are not exhaustive
    """
    test_step("SHIFT", "S", ["ROOT", "the"], ["cat", "sat"], [],
              ("ROOT", "the", "cat"), ("sat",), ())
    test_step("LEFT-ARC", "LA", ["ROOT", "the", "cat"], ["sat"], [],
              ("ROOT", "cat",), ("sat",), (("cat", "the"),))
    test_step("RIGHT-ARC", "RA", ["ROOT", "run", "fast"], [], [],
              ("ROOT", "run",), (), (("run", "fast"),))


def test_parse():
    """Simple tests for the PartialParse.parse function
    Warning: these are not exhaustive
    """
    sentence = ["parse", "this", "sentence"]
    dependencies = PartialParse(sentence).parse(["S", "S", "S", "LA", "RA", "RA"])
    dependencies = tuple(sorted(dependencies))
    expected = (('ROOT', 'parse'), ('parse', 'sentence'), ('sentence', 'this'))
    assert dependencies == expected,  \
        "parse test resulted in dependencies {:}, expected {:}".format(dependencies, expected)
    assert tuple(sentence) == ("parse", "this", "sentence"), \
        "parse test failed: the input sentence should not be modified"
    print("parse test passed!")

In [ ]:
test_parse_step()
test_parse()

## 2. Setup and Preprocessing

In [4]:
from utils.parser_utils import minibatches, load_and_preprocess_data

Preparing data. We will use a subset of Penn Treebank and pretrained embeddings in this task

We are now going to train a neural network to predict, given the state of the stack, buffer, and dependencies, which transition should be applied next. First, the model extracts a feature vector representing the current state. We will be using the feature set presented in the original neural dependency parsing paper: A Fast and Accurate Dependency Parser using Neural Networks. 

The function extracting these features has been implemented for you in parser_utils. This feature vector consists of a list of tokens (e.g., the last word in the stack, first word in the buffer, dependent of the second-to-last word in the stack if there is one, etc.). They can be represented as a list of integers.

In [5]:
parser, embeddings, train_examples, dev_set, test_set = load_and_preprocess_data(True)

Loading data...
took 2.40 seconds
Building parser...
took 0.03 seconds
Loading pretrained embeddings...
took 2.38 seconds
Vectorizing data...
took 0.06 seconds
Preprocessing training data...
took 1.17 seconds


In [6]:
print(len(train_examples), len(dev_set), len(test_set))

48390 500 500


In [7]:
train_examples[10]

([5156,
  660,
  88,
  96,
  85,
  2131,
  5155,
  5155,
  5155,
  5155,
  5155,
  5155,
  91,
  5155,
  113,
  5155,
  5155,
  5155,
  84,
  39,
  40,
  61,
  41,
  39,
  83,
  83,
  83,
  83,
  83,
  83,
  40,
  83,
  41,
  83,
  83,
  83],
 [1, 1, 1],
 2)

In [8]:
embeddings

array([[-0.7292381 , -0.33759412,  0.33548045, ...,  0.5354017 ,
        -0.20434363,  1.0421315 ],
       [-0.4500417 , -1.6892221 ,  0.4156098 , ...,  0.35154536,
        -0.23606075, -2.0565262 ],
       [-0.8837722 , -0.53987306, -0.90973115, ...,  0.21462597,
        -0.12736242,  2.060031  ],
       ...,
       [ 0.6028972 ,  0.05559519,  0.4461097 , ..., -0.5107466 ,
         0.43016586,  1.3869425 ],
       [-0.43125707, -0.30145568, -0.8677901 , ..., -0.23169452,
        -0.506528  ,  0.6272445 ],
       [ 0.39807355,  1.7552437 ,  0.4275137 , ..., -0.21109965,
         1.078214  ,  0.13150327]], dtype=float32)

In [9]:
print(embeddings.shape)

(5157, 50)


Get the full batch of our subset data

In [10]:
minibatch_gen = minibatches(train_examples, len(train_examples))
x_train, y_train = minibatch_gen.__next__()

In [11]:
print(x_train.shape)
print(y_train.shape)

(48390, 36)
(48390, 3)


In [16]:
x_train[0]

array([  88,  241,  245,  339,   87, 5155, 5155, 5155, 5155, 5155, 5155,
       5155, 5155, 5155, 5155, 5155, 5155, 5155,   40,   70,   44,   57,
         46,   83,   83,   83,   83,   83,   83,   83,   83,   83,   83,
         83,   83,   83])

You can use parser.id2tok[word_id] to lookup the word in English.

In [22]:
for word_id in x_train[0]:
  print(parser.id2tok[word_id])

of
how
computers
work
.
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<p>:IN
<p>:WRB
<p>:NNS
<p>:VBP
<p>:.
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>


In [ ]:
y_train[0]

array([0., 1., 0.])

## 3. Model

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dense, Reshape, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

### TODO 3 (Coding):
Build and train a tensroflow keras model to predict an action for each state of of the input. This is a simple classification task. 
- The input and output of the model must match the dimention of x_train and y_train.
- The model must use the provided pretrained embeddings
- The model could comprise of only a feedforward layer and a dropout
- Training loss should be around 0.1 or below, and training categorical_accuracy above 0.94

In [ ]:
model = Sequential()
# Write your code here
pass

In [ ]:
# Write your code here
model.fit()